In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
!pip install fuzzywuzzy

In [0]:
# importing libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
from fuzzywuzzy import fuzz

import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.util import ngrams
from collections import Counter
from nltk import tokenize
import re
import gensim
from gensim.models import Word2Vec
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
# reading the different lexicons and defining negative words
negative_words = ["no", "not", "not any", "none", "never", "neither", "nor", "few", "hardly", "little", "scarcely", "rarely", "seldom", "isn't", "don't", "can't", "hadn't"]

xlsx_file = '/content/drive/My Drive/Sustainable-Peace-Project/BBCLexicon.xlsx'
dfs = pd.read_excel(xlsx_file)
dfs = dfs.drop(['Unnamed: 1','Unnamed: 2','Unnamed: 3'], axis=1)
dfs.columns = ['Peace_Lexicon']
#preprocessing the peace lexicon words 
dfs['Peace_Lexicon'] = dfs['Peace_Lexicon'].str.lower()
dfs['Peace_Lexicon'] = dfs['Peace_Lexicon'].str.replace("_", " ")
dfs['Peace_Lexicon'] = dfs['Peace_Lexicon'].str.strip()
peace_lexicon_list = list(dfs.Peace_Lexicon)

conflict_file = '/content/drive/My Drive/Sustainable-Peace-Project/NewsLexicon-Conflict.csv'
dfs = pd.read_csv(conflict_file)
dfs.columns = ['Conflict_Lexicon']
#preprocessing the conflict lexicon words 
dfs['Conflict_Lexicon'] = dfs['Conflict_Lexicon'].str.lower()
dfs['Conflict_Lexicon'] = dfs['Conflict_Lexicon'].str.replace("_", " ")
dfs['Conflict_Lexicon'] = dfs['Conflict_Lexicon'].str.strip()
conflict_lexicon_list = list(dfs.Conflict_Lexicon)
a
nonPeace_file = '/content/drive/My Drive/Sustainable-Peace-Project/non_peaceful.csv'
dfs = pd.read_csv(nonPeace_file)
dfs.columns = ['NonPeaceful_Lexicon']
#preprocessing the non peace lexicon words
dfs['NonPeaceful_Lexicon'] = dfs['NonPeaceful_Lexicon'].str.lower()
dfs['NonPeaceful_Lexicon'] = dfs['NonPeaceful_Lexicon'].str.replace("_", " ")
dfs['NonPeaceful_Lexicon'] = dfs['NonPeaceful_Lexicon'].str.strip()
nonPeace_lexicon_list = list(dfs.NonPeaceful_Lexicon)

In [0]:
url = 'https://www.bbc.com/news/world-us-canada-52019510'
def bbc_news(url):    
    response = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.120 Safari/537.36'})
    soup = BeautifulSoup(response.content, "lxml")
    article = soup.findAll('p') 
    article = article[12:27]
    news_article = ""
    for a in article:
        temp = a.get_text()
        news_article = news_article + " " + temp
    return news_article
text = bbc_news(url)
text

' How has your life changed because of the coronavirus? The answer will depend a lot on where you live.  Around half the people in the US have been told to stay at home in a bid to try to stop the spread of the virus. That means tens of millions of other Americans are living under less stringent restrictions. Some are still out gallivanting. Many, though, are hunkering down following social distancing guidance.  Here is a look at how some Americans in very different parts of the country are reacting to their own "new normal". Coronavirus cases: 1,247 Restrictions: No state-wide stay-at-home orders but bars and nightclubs are closed and public gatherings over 10 are banned. Making masks for hospital workers Courtney Eidson lives in Sharpsburg, a small rural community south of Atlanta. She\'s a graphic designer and is staying at home with her husband, a commercial appraiser, and her two sons.   She\'s angry at the shortage of protective equipment for those on the frontline of the outbrea

In [0]:
# preprocessing with the text
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[!@#$,"()'']', '', text.lower())
    return(text)
text = preprocess_text(text)
text

" how has your life changed because of the coronavirus? the answer will depend a lot on where you live.  around half the people in the us have been told to stay at home in a bid to try to stop the spread of the virus. that means tens of millions of other americans are living under less stringent restrictions. some are still out gallivanting. many though are hunkering down following social distancing guidance.  here is a look at how some americans in very different parts of the country are reacting to their own new normal. coronavirus cases: 1247 restrictions: no state-wide stay-at-home orders but bars and nightclubs are closed and public gatherings over 10 are banned. making masks for hospital workers courtney eidson lives in sharpsburg a small rural community south of atlanta. she's a graphic designer and is staying at home with her husband a commercial appraiser and her two sons.   she's angry at the shortage of protective equipment for those on the frontline of the outbreak. we've b

In [0]:
# iterate through each sentence in the file 
data = []
for i in sent_tokenize(text): 
    temp = []
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower())
    data.append(temp)
data.append(peace_lexicon_list)

model = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5, sg = 1)

In [0]:
# functions for finding the peace / non-peace / conflict scores - 
def peace_score(text):
    text = text.lower()
    sentences = tokenize.sent_tokenize(text)
    words = []
    temp = []
    for x in sentences:
        temp = x.split(' ')
        words.append(temp)
        temp = []
    flag = 0
    score = 0
    #print(words)
    for x in words:
        for y in x:
            temp = compare_terms(y, peace_lexicon_list)
            if y in negative_words and flag == 0:
                print(y)
                flag = 1
            elif temp != None and flag == 0:
                print(y + ', ' + temp)
                score = score + 1
            elif temp != None and flag == 1:
                print(y + ', ' + temp)
                score = score - 1
                flag = 0
        flag = 0
    return score
def compare_terms(term, lexicon):
    for i in lexicon:
      term = term.strip(' ')
      term = term.strip('.')
      if term != '' and fuzz.ratio(term, i) > 85 and model.similarity(term, i) > 0.1:
        return i

In [0]:
peace_score(text)

no
community, community
healthcare, healthcare
local, local
family, family
help., help


5

In [0]:
peace_score(text)

don't
united, unite
concern, concert


2

In [0]:
peace_score(text)

KeyError: ignored